# Data Quality Assessment

*Dear Thomas,*

*Thank you for sending the three datasets from Sprocket Central Pty Ltd. The table below shows the summary statistics from the three datasets. Please let us know if there are any misunderstandings.*

| **Table name**           |	**No. of records**	| **Distinct Customer IDs** |
|--------------------------|------------------------|---------------------------|
| Customer Demographic     |	4000                | 4000                      |
| Customer Address	       |    20000	            | 3494                      |
| Transaction Data	       |    3999                | 3999                      |

*We discovered several quality issues while reviewing the three datasets you supplied us. We have highlighted mitigations and recommendations to prevent future occurrences of poor data quality and improve the accuracy of the data used for decision-making.*

&emsp;&emsp;***•&emsp;&emsp;Extra Customer_ids are found in both the “Transactions table” and “Customer Address table” but not in the  
&emsp;&emsp;&emsp;&emsp;“Custom Main (Customer Demographic) Table.”***  
&emsp;&emsp;&emsp;&emsp;*Mitigation: Please ensure that the provided tables come from the same time frame. For the analysis, we would be excluding records with the   
&emsp;&emsp;&emsp;&emsp;additional id for the model training.  
&emsp;&emsp;&emsp;&emsp;This is an indication that the data received may not be in sync which each other and it can cause a skew in the analysis if there are missing records in  
&emsp;&emsp;&emsp;&emsp;the data.*

&emsp;&emsp;***•&emsp;&emsp;Stale Data Records are found in the “Transaction Table” which is more than 3 months.***  
&emsp;&emsp;&emsp;&emsp;*Mitigation: Assuming the data was collected in December 2017, the relevant transactions would begin in October 2017, hence we filtered down from   
&emsp;&emsp;&emsp;&emsp;20000 to 5079 records.  
&emsp;&emsp;&emsp;&emsp;Recommendation: Ensure that the transaction table does not contain records for more than 3 months.  
&emsp;&emsp;&emsp;&emsp;Using stale data can affect the predictions of the model and mismatch the needs of the organization.*

&emsp;&emsp;***•&emsp;&emsp;Missing Data in many fields in the Customer Demography and Transactions Table***  
&emsp;&emsp;&emsp;&emsp;*Mitigation: We replaced the missing values of the according to the distribution of the column.Columns that did not have any impact on the analysis   
&emsp;&emsp;&emsp;&emsp;were ignored.  
&emsp;&emsp;&emsp;&emsp;Recommendation: For the customer demographic data, we suggest that these fields are made "compulsory" in the forms the customer is filling in to   
&emsp;&emsp;&emsp;&emsp;capture the data for analysis. Also, ensure that the data pipelines and database are checked thoroughly to prevent missing data for the transaction   
&emsp;&emsp;&emsp;&emsp;data.*  
&emsp;&emsp;&emsp;&emsp;*For the transactions table, we removed the entire records containing missing values in brand, product_line_product_size, standard cost, and   
&emsp;&emsp;&emsp;&emsp;product_first_sold_date since there were no useful data in the entire observations.*

&emsp;&emsp;***•&emsp;&emsp;Inconsistent values for the same attribute and multiple formats for dates in different columns***  
&emsp;&emsp;&emsp;&emsp;***(e.g., New South Wales being represented NSW, Female as “Femal” or “F”)***  
&emsp;&emsp;&emsp;&emsp;*Mitigation: We used regular expressions to rename the states to the abbreviations and the Gender to the full form.  
&emsp;&emsp;&emsp;&emsp;Recommendation:  We suggest that a dropdown menu be used to collect this data for consistency.  
&emsp;&emsp;&emsp;&emsp;This could be caused by either a faulty data pipeline or an open-ended form allowing the customers to freely write their gender or state. Additionally,   
&emsp;&emsp;&emsp;&emsp;we changed the date in the product_first_sold_date column of the transaction data to match the date format of the transaction_date column for &emsp;&emsp;&emsp;&emsp;Consistency.*

&emsp;&emsp;***•&emsp;&emsp;An unusable Column in the Dataset called “default” and a questionable Date of Birth for a Customer***  
&emsp;&emsp;&emsp;&emsp;*Mitigation: Drop the unusable column. Use regular expressions to correct the birth date.  
&emsp;&emsp;&emsp;&emsp;We require more information about this column to deal with the “default” column appropriately. It contains special characters and a lot of   
&emsp;&emsp;&emsp;&emsp;unusable data. We also noticed that a man named Jephthah Bachmann is 179 years old today. We assume this is a mistake and I will change his   
&emsp;&emsp;&emsp;&emsp;birth date from 1843 to 1943 to remove a century from his age.*

*The team would continue with the data cleaning, standardization, and transformation for the model analysis. There would be questions that would arise along the way and all assumptions would be documented. As soon as this exercise is completed, we look forward to spending time with your data SME to address all assumptions and ensure it aligns with Sprocket Central’s understanding.*

*Best Regards,  
Shammah Anucha.  
Data Analyst.*

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# importing files
df = pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name=None, header = 1)

In [3]:
transactions = df["Transactions"] 
nw_customer_list = df['NewCustomerList']
customer_demo = df['CustomerDemographic']
customer_addr = df['CustomerAddress']

transactions= transactions.loc[:,~transactions.columns.str.match("Unnamed")]
transactions.head(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [4]:
nw_customer_list= nw_customer_list.loc[:,~nw_customer_list.columns.str.match("Unnamed")]
nw_customer_list.head(5)

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


In [5]:
customer_demo= customer_demo.loc[:,~customer_demo.columns.str.match("Unnamed")]
customer_demo.head(5)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [6]:
customer_addr= customer_addr.loc[:,~customer_addr.columns.str.match("Unnamed")]
customer_addr.head(5)

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [7]:
print(f"The total number of records in the Customer Demography table is {customer_demo.shape[0]} and the unique customer_id is {customer_demo.customer_id.nunique()}")
print("\n")
print(f"The total number of records in the Transactions table is {transactions.shape[0]} and the unique customer_id is {transactions.customer_id.nunique()}")
print("\n")
print(f"The total number of records in the Customer Address table is {customer_addr.shape[0]} and the unique customer_id is {customer_addr.customer_id.nunique()}")

The total number of records in the Customer Demography table is 4000 and the unique customer_id is 4000


The total number of records in the Transactions table is 20000 and the unique customer_id is 3494


The total number of records in the Customer Address table is 3999 and the unique customer_id is 3999


## Accuracy

In [8]:
customer_demo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [9]:
customer_demo.job_industry_category.value_counts()

Manufacturing         799
Financial Services    774
Health                602
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: job_industry_category, dtype: int64

In [10]:
customer_demo.wealth_segment.value_counts()

Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64

### Accuracy Check for Deceased Indicator

In [11]:
Expected_Deceased_Indicator = 'N'+ '|' + 'Y'
print("Expected Deceased Indicator are: ", Expected_Deceased_Indicator)
print("\n")
if (sum(~customer_demo['deceased_indicator'].str.contains(Expected_Deceased_Indicator, na=False))==0):
    print("✅ - No category apart from N and Y")
else:
    print("❌ The records below do not belong to the expected deceased indicator")
    print(customer_demo[~customer_demo['deceased_indicator'].str.contains(Expected_Deceased_Indicator, na = False)][['deceased_indicator']])

Expected Deceased Indicator are:  N|Y


✅ - No category apart from N and Y


### Accuracy Check for Owns Car

In [12]:
Expected_Label = 'Yes'+ '|' + 'No'
print("Expected Labels are: ", Expected_Label)
print("\n")
if (sum(~customer_demo['owns_car'].str.contains(Expected_Label, na=False))==0):
    print("✅ - No category apart from Yes and No")
else:
    print("❌ The records below do not belong to the expected label")
    print(customer_demo[~customer_demo['owns_car'].str.contains(Expected_Label, na = False)][['owns_car']])

Expected Labels are:  Yes|No


✅ - No category apart from Yes and No


### Accuracy Check for Purchases

In [13]:
print('Accuracy Check for Purchases')
print('\n')
# Check 1
if (sum(customer_demo['past_3_years_bike_related_purchases'].apply(str).str.isdigit()==False)==0):
    print("Check 1: ✅ Purchases has only interger values")
else:
    print("Check 1: ❌ Purchases which do not have Integer Values")
    print(customer_demo[customer_demo['past_3_years_bike_related_purchases'].apply(str).str.isdigit()==False][['first_name','last_name','past_3_years_bike_related_purchases']])

Accuracy Check for Purchases


Check 1: ✅ Purchases has only interger values


### Accuracy Check for Tenure

Check if tenure is float value

### Accuracy Check for Customer_id
Check if the number is from 1-4000

In [14]:
print('Accuracy Check for Customer_id')
print('\n')
# Check 1
customer_demo['customer_id'] = pd.to_numeric(customer_demo['customer_id'], errors='coerce')  # Convert column to numeric values

# Check if the column contains consecutive numbers from 1 to 4000
consecutive_numbers_1_to_4000 = (
    customer_demo['customer_id'].diff().fillna(1) == 1  # Calculate the difference between consecutive numbers
).all()

if consecutive_numbers_1_to_4000:
    print("✅ The column contains consecutive numbers from 1 to 4000.")
else:
    print("❌ The column does not contain consecutive numbers from 1 to 4000.")

Accuracy Check for Customer_id


✅ The column contains consecutive numbers from 1 to 4000.


<ipython-input-14-a74c1e91ca0d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_demo['customer_id'] = pd.to_numeric(customer_demo['customer_id'], errors='coerce')  # Convert column to numeric values


## Transactions

In [15]:
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [16]:
transactions.shape

(20000, 13)

In [17]:
transactions.order_status.value_counts()

Approved     19821
Cancelled      179
Name: order_status, dtype: int64

As seen above, Transations only have two categories Approved and Cancelled

In [18]:
transactions.brand.value_counts()

Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: brand, dtype: int64

In [19]:
transactions.product_line.value_counts()

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: product_line, dtype: int64

As seen above, there are only 4 production lines: Standard, Road, Touring and Mountain

In [20]:
transactions.product_class.value_counts()

medium    13826
high       3013
low        2964
Name: product_class, dtype: int64

As seen above, there are only 3 product classes: Medium, high and low

### Accuracy Check for Customer_id

In [21]:
transactions.customer_id.describe()

count    20000.000000
mean      1738.246050
std       1011.951046
min          1.000000
25%        857.750000
50%       1736.000000
75%       2613.000000
max       5034.000000
Name: customer_id, dtype: float64

In [22]:
print('Accuracy Check for Customer_id')
print('\n')
# Check 1
if (sum(transactions['customer_id'].apply(str).str.isdigit()==False)==0):
    print("Check 1: ✅ Customer_id has only interger values")
    print('\n')
else:
    print("Check 1: ❌ Purchases which do not have Integer Values")
    print(transactions[transactions['customer_id'].apply(str).str.isdigit()==False][['customer_id']])
    print('\n')

# Check 2
temp_df = transactions[transactions['customer_id'].apply(str).str.isdigit()==True]
customer_ids = temp_df['customer_id'].astype(int)
if customer_ids.between(1, 4001).all():
    print("Check 2: ✅ Customer_id is between range in Customer Demographics")
else:
    print("Check 2: ❌ Customer_id not within range are")
    print(temp_df[~customer_ids.between(1,4000)][['customer_id']])

Accuracy Check for Customer_id


Check 1: ✅ Customer_id has only interger values


Check 2: ❌ Customer_id not within range are
       customer_id
8707          5034
16700         5034
17468         5034


In [23]:
print('Accuracy Check for Customer_id')
print('\n')
# Check 1
if (sum(customer_addr['customer_id'].apply(str).str.isdigit()==False)==0):
    print("Check 1: ✅ Customer_id has only interger values")
    print('\n')
else:
    print("Check 1: ❌ Purchases which do not have Integer Values")
    print(customer_addr[customer_addr['customer_id'].apply(str).str.isdigit()==False][['customer_id']])
    print('\n')

# Check 2
temp_df = customer_addr[customer_addr['customer_id'].apply(str).str.isdigit()==True]
customer_ids = temp_df['customer_id'].astype(int)
if customer_ids.between(1, 4001).all():
    print("Check 2: ✅ Customer_id is between range in Customer Demographics")
else:
    print("Check 2: ❌ Customer_id not within range are")
    print(temp_df[~customer_ids.between(1,4000)][['customer_id']])

Accuracy Check for Customer_id


Check 1: ✅ Customer_id has only interger values


Check 2: ❌ Customer_id not within range are
      customer_id
3996         4001
3997         4002
3998         4003


**Solution:** These records would be dropped.


In [24]:
transactions = transactions.loc[transactions.customer_id != 5034]
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0


In [25]:
# checking the online_order column
transactions.online_order.value_counts()

1.0    9829
0.0    9808
Name: online_order, dtype: int64

### Accuracy Check for Date

This section is also for consistency, by changing the format of the date to match the transaction date.

In [26]:
from datetime import datetime, timedelta


formatted_dates = []  # List to store the formatted dates

for date_value in transactions['product_first_sold_date']:
    if pd.notnull(date_value):  # Check if the date_value is not null
        # Convert the date value to a datetime object
        date = datetime(1899, 12, 30) + timedelta(days=int(date_value))

        # Format the date as %y-%m-%d
        formatted_date = date.strftime("%Y-%m-%d")
        formatted_dates.append(formatted_date)
    else:
        formatted_dates.append(np.nan)  # Append NaN for missing values

# Assign the formatted dates to a new column in the DataFrame
transactions['product_first_sold_date_formatted'] = formatted_dates

transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0,2004-08-17


In [27]:
date_diff = pd.DataFrame(pd.to_datetime(transactions.transaction_date) - pd.to_datetime(transactions.product_first_sold_date_formatted))
date_diff['diff'] = date_diff
date_diff

,0,diff
0,1546 days,1546 days
1,1175 days,1175 days
2,6663 days,6663 days
3,6833 days,6833 days
4,783 days,783 days
...,...,...
19995,5087 days,5087 days
19996,7488 days,7488 days
19997,2429 days,2429 days
19998,4703 days,4703 days


Checking if any date in the product_first_sold_date column is after the transaction date

In [28]:
for i in date_diff['diff']:
    if i.days < 0:
        print(i)
else:
    print("No wrong dates")

No wrong dates


## Validity

Drop the unusable column called "default" because of the special characters within it

In [29]:
customer_demo = customer_demo.drop(columns=['default'])
customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,10.0


The age issue should be corrected here. Change the 18.. to 19..

In [30]:
customer_demo['DOB'] = pd.to_datetime(customer_demo['DOB'])
customer_demo['DOB']

0      1953-10-12
1      1980-12-16
2      1954-01-20
3      1961-10-03
4      1977-05-13
          ...    
3995   1975-08-09
3996   2001-07-13
3997          NaT
3998   1973-10-24
3999   1991-11-05
Name: DOB, Length: 4000, dtype: datetime64[ns]

In [31]:
customer_demo['age'] = (pd.to_datetime(datetime.now().strftime("%Y-%m-%d")) - customer_demo['DOB']) // pd.Timedelta(days=365.25)
customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,69.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,42.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,47.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0,22.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,NaN,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


In [32]:
for i in customer_demo['age']:
    if i > 100:
        print(i)


179.0


In [33]:
customer_demo.loc[customer_demo['age']==179]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
33,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,20.0,179.0


In [34]:
customer_demo['DOB'] = customer_demo['DOB'].astype(str).str.replace('1843-12-21','1943-12-21')
customer_demo.loc[customer_demo['age']==179]
# customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
33,34,Jephthah,Bachmann,U,59,1943-12-21,Legal Assistant,IT,Affluent Customer,N,No,20.0,179.0


In [35]:
customer_demo['DOB'] = pd.to_datetime(customer_demo['DOB'])
customer_demo['age'] = (pd.to_datetime(datetime.now().strftime("%Y-%m-%d")) - customer_demo['DOB']) // pd.Timedelta(days=365.25)

In [36]:
customer_addr.property_valuation.value_counts()

9     647
8     646
10    577
7     493
11    281
6     238
5     225
4     214
12    195
3     186
1     154
2     143
Name: property_valuation, dtype: int64

In [37]:
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0,2004-08-17


In [38]:
transactions.standard_cost.describe()

count    19800.000000
mean       556.068047
std        405.976881
min          7.210000
25%        215.140000
50%        507.580000
75%        795.100000
max       1759.850000
Name: standard_cost, dtype: float64

In [39]:
transactions.list_price.describe()

count    19997.000000
mean      1107.919641
std        582.818787
min         12.010000
25%        575.270000
50%       1163.890000
75%       1635.300000
max       2091.470000
Name: list_price, dtype: float64

## Completeness

In [40]:
print('For the customer_demo, we have missing values in the following columns: ', sep='\n')
customer_demo.isna().sum()

For the customer_demo, we have missing values in the following columns: 


customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
age                                     87
dtype: int64

In [41]:
customer_addr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [42]:
print('For the transactions, we have missing values in the following columns: ', sep='\n')
transactions.isna().sum()

For the transactions, we have missing values in the following columns: 


transaction_id                         0
product_id                             0
customer_id                            0
transaction_date                       0
online_order                         360
order_status                           0
brand                                197
product_line                         197
product_class                        197
product_size                         197
list_price                             0
standard_cost                        197
product_first_sold_date              197
product_first_sold_date_formatted    197
dtype: int64

**Solution:** Drop the Missing Values if needed for analysis. The lastname is not necessary, so the values can remain as null, the other columns would be useful. For the Job title, if there is a corresponding job_indusry_category, then there would be no need to remove it.

In [43]:
customer_demo.loc[(customer_demo.job_title.isna()) | customer_demo.job_industry_category.isna()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,8.0,46.0
5,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,Yes,13.0,56.0
6,7,Fina,Merali,Female,6,1976-02-23,NaN,Financial Services,Affluent Customer,N,Yes,11.0,47.0
7,8,Rod,Inder,Male,31,1962-03-30,Media Manager I,NaN,Mass Customer,N,No,7.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,3979,Kleon,Adam,Male,67,1974-07-13,NaN,Financial Services,Mass Customer,N,Yes,18.0,49.0
3982,3983,Jarred,Lyste,Male,19,1965-04-21,Graphic Designer,NaN,Mass Customer,N,Yes,9.0,58.0
3986,3987,Beckie,Wakeham,Female,18,1964-05-29,NaN,Argiculture,Mass Customer,N,No,7.0,59.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


Because the missing values both the job_title and job_industry_category are alot, i would be replacing the misssing values of the job_industry category with the mode of each category

In [44]:
# Assuming 'customer_demo' is the DataFrame containing the customer demographic data

# Calculate the mode of 'job_industry_category' for the entire column
mode_job_industry_category = customer_demo['job_industry_category'].mode().iloc[0]

# Fill missing values in 'job_industry_category' with the mode value
customer_demo['job_industry_category'].fillna(mode_job_industry_category, inplace=True)

In [45]:
customer_demo.loc[(customer_demo.job_title.isna()) | customer_demo.job_industry_category.isna()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,No,7.0,61.0
5,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,Yes,13.0,56.0
6,7,Fina,Merali,Female,6,1976-02-23,NaN,Financial Services,Affluent Customer,N,Yes,11.0,47.0
10,11,Uriah,Bisatt,Male,99,1954-04-30,NaN,Property,Mass Customer,N,No,9.0,69.0
21,22,Deeanne,Durtnell,Female,79,1962-12-10,NaN,IT,Mass Customer,N,No,11.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,3968,Alexandra,Kroch,Female,99,1977-12-22,NaN,Property,High Net Worth,N,No,22.0,45.0
3971,3972,Maribelle,Schaffel,Female,6,1979-03-28,NaN,Retail,Mass Customer,N,No,8.0,44.0
3978,3979,Kleon,Adam,Male,67,1974-07-13,NaN,Financial Services,Mass Customer,N,Yes,18.0,49.0
3986,3987,Beckie,Wakeham,Female,18,1964-05-29,NaN,Argiculture,Mass Customer,N,No,7.0,59.0


Notice that all the missing values for the job_industry_categories have been filled. It's time to do the same for the job_title

In [46]:
# Assuming 'customer_demo' is the DataFrame containing the customer demographic data

# Calculate the mode of 'job_industry_category' for the entire column
mode_job_title = customer_demo['job_title'].mode().iloc[0]

# Fill missing values in 'job_industry_category' with the mode value
customer_demo['job_title'].fillna(mode_job_title, inplace=True)
customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,69.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,42.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,47.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0,22.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,NaN,NaN
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


In [47]:
customer_demo.loc[(customer_demo.job_title.isna()) | customer_demo.job_industry_category.isna()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age


### Completeness Check for DOB

All the missing age would be filled with the average age then the DOB column would be dropped

In [48]:
avg_age = customer_demo['age'].mean()

# Fill missing values in 'age' with the average value
customer_demo['age'].fillna(int(avg_age), inplace=True)
customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,69.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,42.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,47.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0,22.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,NaN,45.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


### Completeness Check for Tenure

In [49]:
# Calculate the mode of 'tenure' for the entire column
mode_tenure = customer_demo['tenure'].mode().iloc[0]

# Fill missing values in 'tenure' with the mode value
customer_demo['tenure'].fillna(mode_tenure, inplace=True)
customer_demo

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,69.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,42.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,47.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0,22.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,7.0,45.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


### Completeness Check for Transactions

In [50]:
# The values here are unsuable
transactions.loc[(transactions.brand.isna())]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
136,137,0,431,2017-09-23,0.0,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaN,NaN
159,160,0,3300,2017-08-27,0.0,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaN,NaN
366,367,0,1614,2017-03-10,0.0,Approved,NaN,NaN,NaN,NaN,850.89,NaN,NaN,NaN
406,407,0,2559,2017-06-14,1.0,Approved,NaN,NaN,NaN,NaN,710.59,NaN,NaN,NaN
676,677,0,2609,2017-07-02,0.0,Approved,NaN,NaN,NaN,NaN,1972.01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19340,19341,0,443,2017-12-26,1.0,Approved,NaN,NaN,NaN,NaN,744.54,NaN,NaN,NaN
19383,19384,0,2407,2017-06-11,0.0,Approved,NaN,NaN,NaN,NaN,1098.18,NaN,NaN,NaN
19793,19794,0,2860,2017-01-13,0.0,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaN,NaN
19859,19860,0,2468,2017-06-24,1.0,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaN,NaN


In [51]:
transactions = transactions.loc[~(transactions.brand.isna())]
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0,2004-08-17


In [52]:
transactions.loc[(transactions.online_order.isna())]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
97,98,49,333,2017-06-23,NaN,Approved,Trek Bicycles,Road,medium,medium,533.51,400.13,37823.0,2003-07-21
166,167,90,3177,2017-04-26,NaN,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41,38482.0,2005-05-10
169,170,6,404,2017-10-16,NaN,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,37838.0,2003-08-05
250,251,63,1967,2017-04-11,NaN,Approved,Solex,Standard,medium,medium,1483.20,99.59,42145.0,2015-05-21
300,301,78,2530,2017-03-24,NaN,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,35455.0,1997-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19514,19515,51,690,2017-01-22,NaN,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0,2003-07-21
19573,19574,18,1735,2017-01-15,NaN,Approved,Solex,Standard,medium,medium,575.27,431.45,41345.0,2013-03-12
19580,19581,49,1933,2017-10-12,NaN,Approved,Trek Bicycles,Road,medium,medium,533.51,400.13,41064.0,2012-06-04
19635,19636,98,1389,2017-07-26,NaN,Approved,Trek Bicycles,Standard,high,medium,358.39,215.03,38002.0,2004-01-16


Replace the missing values with the mode

In [53]:
# Calculate the mode of 'online_order' for the entire column
mode_online_order = transactions['online_order'].mode().iloc[0]

# Fill missing values in 'tenure' with the mode value
transactions['online_order'].fillna(mode_online_order, inplace=True)
transactions.head()

C:\Users\user\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2015-08-10


## Consistency

### Consistency Check for All Columns

In [54]:
columns = ['gender', 'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car', 'country']
for i in columns:
    print(i)
    print(nw_customer_list[i].value_counts(), sep='\n')
    print('\n')

gender
Female    513
Male      470
U          17
Name: gender, dtype: int64


job_industry_category
Financial Services    203
Manufacturing         199
Health                152
Retail                 78
Property               64
IT                     51
Entertainment          37
Argiculture            26
Telecommunications     25
Name: job_industry_category, dtype: int64


wealth_segment
Mass Customer        508
High Net Worth       251
Affluent Customer    241
Name: wealth_segment, dtype: int64


deceased_indicator
N    1000
Name: deceased_indicator, dtype: int64


owns_car
No     507
Yes    493
Name: owns_car, dtype: int64


country
Australia    1000
Name: country, dtype: int64




In [55]:
columns = ['online_order', 'order_status', 'brand', 'product_line', 'product_class', 'product_size',]
for i in columns:
    print(i)
    print(transactions[i].value_counts(), sep='\n')
    print('\n')

online_order
1.0    10097
0.0     9703
Name: online_order, dtype: int64


order_status
Approved     19622
Cancelled      178
Name: order_status, dtype: int64


brand
Solex             4252
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3042
Trek Bicycles     2990
Norco Bicycles    2909
Name: brand, dtype: int64


product_line
Standard    14175
Road         3968
Touring      1234
Mountain      423
Name: product_line, dtype: int64


product_class
medium    13823
high       3013
low        2964
Name: product_class, dtype: int64


product_size
medium    12987
large      3976
small      2837
Name: product_size, dtype: int64




In [56]:
columns = ['gender', 'job_industry_category', 'wealth_segment', 'deceased_indicator']
for i in columns:
    print(i)
    print(customer_demo[i].value_counts(), sep='\n')
    print('\n')

gender
Female    2037
Male      1872
U           88
Femal        1
M            1
F            1
Name: gender, dtype: int64


job_industry_category
Manufacturing         1455
Financial Services     774
Health                 602
Retail                 358
Property               267
IT                     223
Entertainment          136
Argiculture            113
Telecommunications      72
Name: job_industry_category, dtype: int64


wealth_segment
Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64


deceased_indicator
N    3998
Y       2
Name: deceased_indicator, dtype: int64




In [57]:
customer_addr.columns

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')

### Consistency Check for Gender

In [58]:
Expected_Gender = 'Female'+ '|' + 'Male' + '|' + 'U'
print("Expected Gender are: ", Expected_Gender)
print("\n")
if (sum(~customer_demo['gender'].str.contains(Expected_Gender, na=False))==0):
    print("✅ - No category apart from Female, Male, and U")
else:
    print("❌ The records below do not belong to the expected gender")
    print(customer_demo[~customer_demo['gender'].str.contains(Expected_Gender, na = False)][['gender']])

Expected Gender are:  Female|Male|U


❌ The records below do not belong to the expected gender
   gender
0       F
53  Femal
56      M


**Solution:** Convert the discrepancies to Female and Male. 

In [59]:
customer_demo['gender'] = customer_demo['gender'].str.replace('Femal$','Female', regex= True)
customer_demo['gender'] = customer_demo['gender'].str.replace('F$','Female', regex= True)
customer_demo['gender'] = customer_demo['gender'].str.replace('M$','Male', regex= True)
customer_demo.gender.value_counts()

Female    2039
Male      1873
U           88
Name: gender, dtype: int64

In [60]:
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0,2004-08-17


The product_first_sold_date should be in a datetime format. I will drop the former format

In [61]:
transactions = transactions.drop(columns=['product_first_sold_date'])
transactions

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-17


In [62]:
transactions.list_price.describe()

count    19800.000000
mean      1108.088981
std        582.763905
min         12.010000
25%        575.270000
50%       1163.890000
75%       1635.300000
max       2091.470000
Name: list_price, dtype: float64

In [63]:
customer_addr

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7


In [64]:
for i in customer_addr.postcode:
    if len(str(i)) > 4:
        print(i)
else:
    print("All pstcode format are correct")

All pstcode format are correct


### Consistency Check for state

In [65]:
customer_addr.state.value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

**Solution:** For the state column to be consistent, one value should be chosen. Convert Victoria to VIC and  New South Wales to NSW

In [66]:
customer_addr['state'] = customer_addr['state'].str.replace('New South Wales','NSW', regex= True)
customer_addr['state'] = customer_addr['state'].str.replace('Victoria','VIC', regex= True)
customer_addr.state.value_counts()

<ipython-input-66-4e164254e72a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_addr['state'] = customer_addr['state'].str.replace('New South Wales','NSW', regex= True)
<ipython-input-66-4e164254e72a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_addr['state'] = customer_addr['state'].str.replace('Victoria','VIC', regex= True)


NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

### Consistency Check for Address

In [67]:
import re
# Check if address number comes last instead of first
pattern = r'\d+$'  # Regular expression pattern to match the desired format

# Check if the column values match the pattern
matches_pattern = customer_addr['address'].astype(str).str.contains(pattern)

# Print the rows that do not match the pattern
print(customer_addr[matches_pattern])


Empty DataFrame
Columns: [customer_id, address, postcode, state, country, property_valuation]
Index: []


## Uniqueness

In [68]:
customer_demo.drop_duplicates()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,69.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,42.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,69.0
3,4,Talbot,NaN,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0,61.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,N,No,19.0,47.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,1.0,22.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,N,No,7.0,45.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,N,Yes,10.0,49.0


In [69]:
transactions.drop_duplicates()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date_formatted
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-17


In [70]:
customer_addr.drop_duplicates()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,NSW,Australia,10
1,2,6 Meadow Vale Court,2153,NSW,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,NSW,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7


In [71]:
nw_customer_list.drop_duplicates()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,996,0.374000
996,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,997,0.357000
997,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,997,0.357000
998,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,997,0.357000


## Relevance

In [72]:
year_check = transactions['transaction_date'].dt.strftime('%Y')
year_check.describe()

count     19800
unique        1
top        2017
freq      19800
Name: transaction_date, dtype: object

In [73]:
month_check = transactions['transaction_date'].dt.strftime('%m')
month_check.value_counts()

10    1750
08    1735
07    1700
05    1666
01    1663
11    1648
04    1641
12    1633
03    1629
02    1612
06    1564
09    1559
Name: transaction_date, dtype: int64

Assuming the dataset was collected in from December, 2017. The relevent transactions would be from October making it 3 months.

In [74]:
transactions['transaction_month'] = transactions['transaction_date'].dt.strftime('%m')
transactions_3months = transactions.loc[(transactions['transaction_month']>="10")& (transactions['transaction_month']<="12")]
transactions_3months

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date_formatted,transaction_month
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,10
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,10
15,16,3,2961,2017-10-10,0.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2003-09-09,10
20,21,27,1582,2017-10-09,0.0,Approved,Trek Bicycles,Standard,medium,medium,499.53,388.72,1999-06-23,10
23,24,82,515,2017-10-18,0.0,Approved,Giant Bicycles,Road,medium,medium,1538.99,829.65,2016-02-04,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19984,19985,31,873,2017-11-15,1.0,Approved,Giant Bicycles,Standard,medium,medium,230.91,173.18,1994-07-12,11
19985,19986,36,2918,2017-11-09,1.0,Approved,Solex,Standard,low,medium,945.04,507.58,2008-03-19,11
19986,19987,38,2385,2017-11-01,0.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2011-03-16,11
19993,19994,77,2618,2017-12-23,0.0,Approved,Norco Bicycles,Road,medium,large,1240.31,795.10,2011-01-10,12
